In [ ]:
# Recommendation Algo

In [1]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
nlp = spacy.load("en_core_web_sm")
#only run on first time to download the pretrained model and then extract it.

# import gensim.downloader as api
# import gzip
# path = api.load("word2vec-google-news-300", return_path=True)
# print(path)
# def decompress_gz(input_path, output_path):
#     with gzip.open(input_path, 'rb') as f_in:
#         file_content = f_in.read()
#         with open(output_path, 'wb') as f_out:
#             f_out.write(file_content)

# decompress_gz(path, path[:-2]+"bin")

path = r'llama\word2vec-google-news-300.bin'
model = KeyedVectors.load_word2vec_format(path, binary=True)
def sentanceSimplifier(txt): 
    doc = nlp(txt)
    out = ""
    for token in doc:
        if token.pos_ in ['PROPN', 'VERB','NUM', 'NOUN']:
            out+=token.lemma_+" "
    return out[:-1]



In [2]:
abstract = 'From scary stories to horror films and haunted houses, the horror genre is wildly popular. Although horror aims to elicit fear and anxiety, many people with anxiety are horror fans and some report using horror to cope with anxiety. In this article, we provide a theoretical rationale for why people with anxiety might choose to access and find relief in horror films. First, we discuss aspects of horror that could make it alluring to people with anxiety and how the use of horror may be negatively reinforcing. Next, we examine how engagement with horror could be used to build skills for resilience in generalized situations. We build on processes from evidence-based therapies (i.e., cognitive behavioral therapy and exposure therapy) to explain how horror media has the potential to be used as a therapeutic tool. Finally, we discuss steps for future research on horror as a therapeutic tool for anxiety-related disorders.'
abstract2 = 'Intriguingly, the horror genre, renowned for evoking fear and trepidation, has garnered immense popularity across various media forms, from spine-chilling tales to unsettling cinematic experiences and spine-tingling haunted attractions. Paradoxically, amidst its spine-tingling narratives and heart-pounding scares, a noteworthy phenomenon emerges: individuals grappling with anxiety often find solace and intrigue within the realm of horror. This article endeavors to elucidate the theoretical underpinnings that underlie the inclination of individuals with anxiety towards engaging with horror content, delving into the potential reasons behind this intriguing connection.'
#first 2 are similar 3 is different
abstract3 = 'That there was variation in the theories expounded by the major early thinkers on evolution is well established. What is less effectively articulated is how their intellectual ambitions and approaches to interpreting evidence reflect a deep-rooted difference in their views on what sort of systems, and problems evolution deals with; insights which speak directly to contemporary debates about how we communicate the nature of science with consequences for learning practice. We provide a re-reading of the perspectives adopted by those who formulated the central concepts of evolutionary theory to expose slender but persistent differences of emphasis. Our analysis contrasts the approaches to enquiry of three voyaging naturalists; Charles Darwin, Thomas Henry Huxley, and Alfred Russel Wallace. This potted history is used to illustrate the relationship between different traditions of scientific enquiry, specifically the natural philosophy approach characteristic of Wallace’s work and the later Neo-Darwinists; Huxley’s more empirical, natural history approach that anticipated punctuated equilibria, and Darwin’s emphasis on agency and choice, which suggests a more humanistic mind-set. These understandings are used to propose a model of the structure of evolutionary science which has broader value as a motif for exploring and communicating the transitions between different modes of enquiry.'
test = sentanceSimplifier(abstract).split(" ")
test

In [3]:
print(len(abstract))

926


In [ ]:
def sentace2vec(txt):
    simplified_txt = sentanceSimplifier(txt)
    vecs = []
    for word in simplified_txt.split(" "):
        vecs.append(model[word])
    vecs_arr = np.array(vecs)
    return  vecs_arr.sum(axis=0)
%timeit sentace2vec(abstract)


In [ ]:
#can look to see if there is a faster way to implement this or if this is good enough
def sentace2vec(txt):
    simplified_txt = sentanceSimplifier(txt)
    words = simplified_txt.split(" ")
    return np.array([model[word] for word in words if word in model]).sum(axis=0)/len(words)
#%timeit sentace2vec(abstract) #30.8 ms ± 4.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
sentace2vec(abstract)

# this looks pretty good IMO mostly diff of < 3% so should be good to go



In [ ]:
sentace2vec(abstract) - sentace2vec(abstract2)
similarity = cosine_similarity([sentace2vec(abstract)], [sentace2vec(abstract2)])[0][0]
#another way to compare the 2 sentances
similarity


In [ ]:
similarity = cosine_similarity([sentace2vec(abstract)], [sentace2vec(abstract3)])[0][0]
# downside is that they might have a small range but I think that is fine, since the training set is all words 
# and we are only using scientific words there is higher similarity even if they are different I think 
similarity

In [ ]:
# so I think at this point we hold a vector and a weight for each and then we find similarity,
# see the similarity weighted value and then order by that